# Chinese Word Segmentation

## Download the data and Prepare training/test data

In [ ]:
import requests

remote_url = "https://raw.githubusercontent.com/hhhuang/nlp2019fall/master/word_segmentation/"

r = requests.get(remote_url + "data/as_training.utf8", allow_redirects=True)
open('as_training.utf8', 'wb').write(r.content)

r = requests.get(remote_url + "data/as_testing_gold.utf8", allow_redirects=True)
open('as_testing_gold.utf8', 'wb').write(r.content)


942571

In [ ]:
raw_train = []
raw_test = []
with open("as_training.utf8", encoding="utf8") as fin:
    for line in fin:
        raw_train.append(line.strip().split("　"))   # It is a full white space

with open("as_testing_gold.utf8", encoding="utf8") as fin:
    for line in fin:
        raw_test.append(line.strip().split("　"))   # It is a full white space

print("Number of sentences in the training data: %d" % len(raw_train))
print("Number of sentences in the test data: %d" % len(raw_test))

Number of sentences in the training data: 708953
Number of sentences in the test data: 14432


## Use jieba

In [ ]:
!pip install jieba

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import jieba

print(list(jieba.cut("".join(raw_test[0]))))

Building prefix dict from the default dictionary ...
DEBUG:jieba:Building prefix dict from the default dictionary ...
Dumping model to file cache /tmp/jieba.cache
DEBUG:jieba:Dumping model to file cache /tmp/jieba.cache
Loading model cost 0.702 seconds.
DEBUG:jieba:Loading model cost 0.702 seconds.
Prefix dict has been built successfully.
DEBUG:jieba:Prefix dict has been built successfully.


['許多', '社區長', '青學苑', '多', '開設', '有書法', '、', '插花', '、', '土風', '舞班', '，']


## Perform jieba on Simplified Chinese and restore the segmetnation to the original input.

In [ ]:
!pip install hanziconv

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 276.8/276.8 kB 9.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for hanziconv: filename=hanziconv-0.3.2-py2.py3-none-any.whl size=23224 sha256=0366cc878c10988219832f245ab061bb702532a2a22c2c3b48fd39e918f4506b
  Stored in directory: /root/.cache/pip/wheels/62/5a/e5/779585d31b977c0fa445b649a256d1fae60728fdc21c1373dc
Successfully built hanziconv


In [ ]:
from hanziconv.hanziconv import HanziConv

print(list(jieba.cut(HanziConv.toSimplified("".join(raw_test[0])))))

['许多', '社区', '长青', '学苑', '多', '开设', '有', '书法', '、', '插花', '、', '土风舞', '班', '，']


In [ ]:
def restore(text, toks):
    results = []
    offset = 0
    for tok in toks:
        results.append(text[offset:offset + len(tok)])
        offset += len(tok)
    return results

text = "".join(raw_test[0])
print(restore(text, list(jieba.cut(HanziConv.toSimplified(text)))))

['許多', '社區', '長青', '學苑', '多', '開設', '有', '書法', '、', '插花', '、', '土風舞', '班', '，']


## Build Your Own Chinese Word Segmenter

Prepare training instances for sequence labeling by converting a list of words to a sequence of tags

Create a CRF model for word segmentation

In [ ]:
!pip install sklearn-crfsuite

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


##Assignment II

Modify the following block ONLY for Assignment II

In [ ]:
import sklearn_crfsuite
from sklearn_crfsuite import scorers, metrics

def words_to_tags(words):
    tags = []
    for word in words:
        if len(word) == 1:
            tags.append('S')
        else:
            for i in range(len(word)):
                if i == 0:
                    tags.append('L')
                elif i == len(word) - 1:
                    tags.append('R')
                else:
                    tags.append('M')
    return tags

train_X = []
train_Y = []

test_X = []
test_Y = []

for sent in raw_train:
    train_X.append(list("".join(sent)))  # Make the unsegmented sentence as a sequence of characters
    train_Y.append(words_to_tags(sent))

for sent in raw_test:
    test_X.append(list("".join(sent)))  # Make the unsegmented sentence
    test_Y.append(words_to_tags(sent))

print(test_X[0])
print(test_Y[0])

def extract_sent_features(x):
    sent_features = []
    for i in range(len(x)):
        sent_features.append(extract_char_features(x, i))
    return sent_features

def extract_char_features(sent, position):
    char_features = {}
    for i in range(-3, 4):
        if len(sent) > position + i >= 0:
            char_features['char_at_%d' % i] = sent[position + i]
    return char_features

crf_tagger = sklearn_crfsuite.CRF(algorithm='lbfgs', min_freq=20, max_iterations=300, verbose=True)

feature_X = []
for x in train_X:
    feature_X.append(extract_sent_features(x))
try:
    crf_tagger.fit(feature_X, train_Y)
except AttributeError:
    pass

['許', '多', '社', '區', '長', '青', '學', '苑', '多', '開', '設', '有', '書', '法', '、', '插', '花', '、', '土', '風', '舞', '班', '，']
['L', 'R', 'L', 'R', 'L', 'R', 'L', 'R', 'S', 'L', 'R', 'S', 'L', 'R', 'S', 'L', 'R', 'S', 'L', 'M', 'M', 'R', 'S']


loading training data to CRFsuite: 100%|██████████| 708953/708953 [00:46<00:00, 15313.52it/s]



Feature generation
type: CRF1d
feature.minfreq: 20.000000
feature.possible_states: 0
feature.possible_transitions: 0
0....1....2....3....4....5....6....7....8....9....10
Number of features: 70338
Seconds required: 11.302

L-BFGS optimization
c1: 0.000000
c2: 1.000000
num_memories: 6
max_iterations: 300
epsilon: 0.000010
stop: 10
delta: 0.000010
linesearch: MoreThuente
linesearch.max_iterations: 20

Iter 1   time=6.24  loss=9523529.41 active=70226 feature_norm=1.00
Iter 2   time=3.21  loss=5960274.10 active=70338 feature_norm=5.08
Iter 3   time=6.41  loss=5632639.26 active=70338 feature_norm=5.60
Iter 4   time=3.19  loss=5317692.30 active=70338 feature_norm=6.39
Iter 5   time=3.22  loss=3919265.17 active=70338 feature_norm=12.31
Iter 6   time=3.20  loss=3640476.35 active=70338 feature_norm=14.54
Iter 7   time=3.19  loss=3411889.18 active=70338 feature_norm=14.45
Iter 8   time=3.18  loss=3244375.69 active=70338 feature_norm=16.20
Iter 9   time=3.18  loss=3037585.23 active=70338 feature_

In [ ]:
def segment(sent):
    tags = crf_tagger.predict_single(extract_sent_features(list(sent)))
    tokens = []
    tok = ""
    for ch, tag in zip(list(sent), tags):
        if tag in ['S', 'L'] and tok != "":
            tokens.append(tok)
            tok = ""
        tok += ch
    if tok:
        tokens.append(tok)
    return tokens

print(segment("法國總統馬克宏已到現場勘災，初步傳出火警可能與目前聖母院的維修工程有關。"))

['法國', '總統', '馬克宏', '已到', '現場', '勘災', '，', '初步', '傳出', '火警', '可能', '與', '目前', '聖母院', '的', '維修', '工程', '有關', '。']


## Evaluation
Scorer for CWS

In [ ]:
def compare(actual_toks, pred_toks):
    i = 0
    j = 0
    p = 0
    q = 0
    tp = 0
    fp = 0
    while i < len(actual_toks) and j < len(pred_toks):
        if p == q:
            if actual_toks[i] == pred_toks[j]:
                tp += 1
            else:
                fp += 1
            p += len(actual_toks[i])
            q += len(pred_toks[j])
            i += 1
            j += 1
        elif p < q:
            p += len(actual_toks[i])
            i += 1
        else:
            fp += 1
            q += len(pred_toks[j])
            j += 1
    return tp, fp, len(actual_toks)

def score(actual_sents, pred_sents):
    tp = 0
    fp = 0
    total = 0
    for actual_toks, pred_toks in zip(actual_sents, pred_sents):
        tp_, fp_, total_ = compare(actual_toks, pred_toks)
        tp += tp_
        fp += fp_
        total += total_
    recall = float(tp) / total
    precision = float(tp) / (tp + fp)
    f1 = 2.0 * recall * precision / (recall + precision)
    return recall, precision, f1

Testing

In [ ]:
pred = []
actual = []
for sent in raw_test:
    pred.append(segment("".join(sent)))
    actual.append(sent)
print(actual[0])
print(pred[0])

print(score(actual, pred))

['許多', '社區', '長青', '學苑', '多', '開設', '有', '書法', '、', '插花', '、', '土風舞班', '，']
['許多', '社區長', '青學', '苑多', '開設', '有', '書法', '、', '插花', '、', '土風舞班', '，']
(0.8590069811443857, 0.8518857013450231, 0.8554315207283446)


In [ ]:
pred = []
actual = []
fout = open("jieba.out", "w")
for sent in raw_test:
    text = "".join(sent)
    r = list(jieba.cut(HanziConv.toSimplified(text)))
    r = restore(text, r)
    fout.write(" ".join(r) + "\n")
    pred.append(r)
    actual.append(sent)
print(actual[0])
print(pred[0])

print(score(actual, pred))

['許多', '社區', '長青', '學苑', '多', '開設', '有', '書法', '、', '插花', '、', '土風舞班', '，']
['許多', '社區', '長青', '學苑', '多', '開設', '有', '書法', '、', '插花', '、', '土風舞', '班', '，']
(0.8149262738957396, 0.8293466352378739, 0.8220732208967503)


In [ ]:
!pip install simpletransformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.7/250.7 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 kB 35.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 107.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 74.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 94.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 6.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 104.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 104.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 27.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 kB 24.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━

In [ ]:
import pandas as pd

def prepare_data(X, Y, limit=None):
    data = []
    for sid, (x, y) in enumerate(zip(X, Y)):
        for x_tok, y_tok in zip(x, y):
            data.append([sid, x_tok, y_tok])
        if limit and sid >= limit:
            break
    return data

train_data = prepare_data(train_X, train_Y, 3000)
eval_data = prepare_data(test_X, test_Y, 100)
print(train_data[:10])
print(eval_data[:10])

train_data = pd.DataFrame(train_data, columns=["sentence_id", "words", "labels"])
eval_data = pd.DataFrame(eval_data, columns=["sentence_id", "words", "labels"])

[[0, '時', 'L'], [0, '間', 'R'], [0, '：', 'S'], [1, '三', 'L'], [1, '月', 'R'], [1, '十', 'L'], [1, '日', 'R'], [1, '（', 'S'], [1, '星', 'L'], [1, '期', 'M']]
[[0, '許', 'L'], [0, '多', 'R'], [0, '社', 'L'], [0, '區', 'R'], [0, '長', 'L'], [0, '青', 'R'], [0, '學', 'L'], [0, '苑', 'R'], [0, '多', 'S'], [0, '開', 'L']]


In [ ]:
from simpletransformers.ner import NERModel, NERArgs

model_args = NERArgs()
model_args.train_batch_size = 16
model_args.evaluate_during_training = True
model_args.labels_list = ["L", "M", "R", "S"]
model_args.overwrite_output_dir = True

model = NERModel("bert", "bert-base-chinese", args=model_args)
model.train_model(train_data, eval_data=eval_data)
result, model_outputs, preds_list = model.eval_model(eval_data)

Some weights of the model checkpoint at bert-base-chinese were not used when initializing BertForTokenClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-c

  0%|          | 0/7 [00:00<?, ?it/s]

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Running Epoch 0 of 1:   0%|          | 0/188 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/13 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/13 [00:00<?, ?it/s]

In [ ]:
predictions, raw_outputs = model.predict([" ".join(list("今天天氣晴朗"))])
print(predictions)
predictions, raw_outputs = model.predict([" ".join(list("天朗氣清、惠風和暢"))])
print(predictions)

  0%|          | 0/1 [00:00<?, ?it/s]

Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

[[{'今': 'L'}, {'天': 'R'}, {'天': 'L'}, {'氣': 'R'}, {'晴': 'L'}, {'朗': 'R'}]]


  0%|          | 0/1 [00:00<?, ?it/s]

Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

[[{'天': 'L'}, {'朗': 'M'}, {'氣': 'M'}, {'清': 'R'}, {'、': 'S'}, {'惠': 'S'}, {'風': 'M'}, {'和': 'R'}, {'暢': 'S'}]]


In [ ]:
def bert_segment(sents):
    for i in range(len(sents)):
        sents[i] = " ".join(list(sents[i]))

    predictions, _ = model.predict(sents)
    sent_tokens = []
    for prediction in predictions:
        tokens = []
        tok = ""
        for kv in prediction:
            for ch, tag in kv.items():
                if tag in ['S', 'L'] and tok != "":
                    tokens.append(tok)
                    tok = ""
                tok += ch
        if tok:
            tokens.append(tok)
        sent_tokens.append(tokens)
    return sent_tokens

print(bert_segment(["法國總統馬克宏已到現場勘災，初步傳出火警可能與目前聖母院的維修工程有關。"]))

  0%|          | 0/1 [00:00<?, ?it/s]

Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

[['法國', '總統', '馬克宏', '已', '到', '現場', '勘災', '，', '初步', '傳出', '火警', '可能', '與', '目前', '聖母院', '的', '維修', '工程', '有關', '。']]


In [ ]:
pred = bert_segment(["".join(sent) for sent in raw_test])
actual = raw_test
print(actual[0])
print(pred[0])
print(score(actual, pred))

  0%|          | 0/21 [00:00<?, ?it/s]

Running Prediction:   0%|          | 0/1804 [00:00<?, ?it/s]

['許多', '社區', '長青', '學苑', '多', '開設', '有', '書法', '、', '插花', '、', '土風舞班', '，']
['許多', '社區', '長青', '學苑', '多', '開設', '有', '書法', '、', '插花', '、', '土風舞班', '，']
(0.9398854961832062, 0.9365400555853528, 0.9382097936255953)


In [ ]:
text = """勞資評斷委員會，昨日下午假社會局會議室舉行第八次會議。出席吳開先，潘公展，張維，杜月笙（朱文德代），宣鐵吾（黃東昇代），水祥雲，黃仁貴，佘敬成，吳蘊初（田和卿代）。列席李劍華，劉靖　，樊振邦等。因生活指數凍結後，二月份實行差額貼補制度，規定每名貼補七萬三千元。各工廠公會甚為關懷，紛紛請求社會局解釋。惟因無統一辦法，恐以後引起糾紛，故商討決定加工等發給差金辦法，細則為：（一）件工，無論工人生產量之多寡，概以工作日數作為發給差金之標準。（二）童工（包括學徒藝徒養成工等，）凡已免費供給膳食者，其米油鹽煤之差金不予發給，祗發給布糖兩項。凡不供膳者，給予三分之一差金。（三）職工請請假工資照扣者，其差金比例照扣。但病假不扣差金。此外，習慣上向不扣除工資者，差金亦不扣除。（四）各工廠公司行號解雇職工，依照評斷會辦法發給遺散費者，應自二月份起將差金併入工資內發給之。（五）延長工作時間以及臨時加工，不再發給差金。（六）各廠過去免費發給食米者，差金中食米一項不予扣除。（七）工資多寡差金不加區別。另西服時裝等業工人在旺月淡月及停工等情形下，差金應如何發給，尚待研究。"""
gold = """勞資 評斷 委員會 ， 昨日 下午 假 社會局 會議室 舉行 第八 次 會議 。
出席 吳開先 ， 潘公展 ， 張維 ， 杜月笙（朱文德代）， 宣鐵吾（黃東昇代） ， 水祥雲 ， 黃仁貴 ， 佘敬成 ， 吳蘊初（田和卿代） 。
列席 李劍華 ， 劉靖 ， 樊振邦 等 。
因 生活 指數 凍結 後 ， 二月份 實行差額 貼補 制度 ， 規定 每 名 貼補 七萬三千元 。
各 工廠 公會 甚 為 關懷 ， 紛紛 請求 社會局 解釋 。
惟因 無 統一 辦法 ， 恐 以後 引起 糾紛 ， 故 商討 決定 加工 等 發給 差金 辦法 ， 細則 為 ：
（ 一 ）件工，無論 工人 生產量 之 多寡 ， 概 以 工作 日數 作 為 發給 差金 之標準 。
（ 二 ）童工（ 包括 學徒 藝徒 養成工 等 ， ）凡 已 免費 供給 膳食 者 ， 其 米 油 鹽 煤 之 差金 不予 發給 ，
祗 發給 布 糖 兩項 。
凡 不 供膳者 ， 給予 三分之一 差金 。
（ 三 ）職工 請 請假 工資 照扣 者 ， 其 差金 比例照扣 。
但 病假 不 扣 差金 。
此外 ， 習慣 上 向 不 扣除 工資 者 ， 差金 亦 不 扣除 。
（ 四 ）各 工廠 公司 行號 解雇 職工 ， 依照 評斷會 辦法 發給 遺散費 者 ， 應 自 二月份 起 將 差金 併入 工資 內 發給 之 。
（ 五 ）延長 工作 時間 以及 臨時 加工 ， 不再 發給 差金 。
（ 六 ）各 廠 過去 免費 發給 食米 者 ， 差金 中 食米 一項 不予 扣除 。
（ 七 ）工資 多寡 差金 不加 區別 。
另 西服 時裝 等 業 工人 在 旺月 淡月 及 停工 等 情形 下 ， 差金 應 如何 發給 ，尚 待 研究。"""


text = "寗紹商輪公司於二十四日午後二時半在寗波旅滬同鄕會開第十八屆股東年會到會股東計一萬二千二百七十七股計九千一百二十一權方椒伯主席袁經理報告丙寅年營業狀况監察人謝蓮卿報告丙寅年帳略次股東票選董事監察人開票結果當留舊董事三人方椒伯·方樵苓·樂振葆·新選董事八人何謀軒·謝蘅牕·孫梅堂·王心貫·樓恂如·何積璠·傅品圭·徐棣蓀·候補董事十一人袁履登·屠康侯·項松茂·洪賢鈁·洪賢賓·何紹庭·李志方·謝蓮卿·王雲甫·施才高·李詠裳監察人五人石運乾·謝蓮卿·洪賢鈁·周仰山·洪雁賓·候補監察人五人王雲甫·傅其霖·樓恂如·劉鴻生·項松茂旋即散會"
text = "七月初一日福建正主考葛寳華副主考黄紹第今晨過蘇赴閩太倉州蔣禀到同知羅承恩禀謝奉委閔行釐局差卽辭並銷發審局差新選建始縣逃後辭行赴湖北代理南匯縣楊十晟禀辭赴任知縣吳俟卿禀銷南路總巡差又錢國選禀知到南路總巡差　徐樹森禀知赴寗見制憲卽辭巡檢沈廷楨禀銷吳淞江釐局司員差典史薛觀禀知奉委盤門外帮同彈壟差○初三日同知銜湖北候補知縣徐元瀛浙江人由鄂來通判陳寳樽謝委盛澤厘局差委赴四川守提償欵委員候補知縣鹿　理禀辭赴川圓　門外二叚巡查委員候補縣丞竇以　禀知代辦城外陸路總巡差道庫大使許光熙謝委荆溪縣保甲差卽辭主簿郝炳麟禀知奉委文闈執事差巡檢何天綬謝委　門旱關差"
text = "新聞紙之流布於寰區也香港則間日呈奇峙佳名於三秀滬瀆則每晨抽秘鬥彩筆於兩家或著錄應來復之期教堂握槧或成帙在合朔之候京國傳書凡此紀事而纂言莫不標新而領異議分鄉校願考見夫濟世之經猷源討楹書願輯夫證今之學問下至方言里語足備談諧雜筆小詩亦供嗢噱奈日力之有限致篇幅之無多花類折枝僅悦一時之目玉非全壁誰知千古之心斷爛之朝錄堪聞見之錄難遍用特勤加搜討遍訪知交積三十日之斷錦零縑居然成幅合四大洲之隋珠和璧用示奇珍擬為瀛寰瑣紀一書凡巳登申報者不錄每月以朔日出書一卷其價則每本八十文若本館躉售則每本六十五文初印之時以二千本為率如有各書坊定印必當預先知會臨時不能議添矣備中朝之史料名敢託夫官廣異域之談資陋不嫌夫蠻語瑣聞兼述田附搜神志怪之餘碎事同登不薄巷議街談之末所願文壇健者儒林丈人惠贈瑶章共襄盛舉庶幾琳瑯日耀如入寶山梨棗風行不慚詞苑則本館有厚望焉"
pred = bert_segment([text])
actual = [gold.replace("\n", " ").split(" ")]
print(" ".join(pred[0]))
#print(actual[0])
#print(score(actual, pred))


  0%|          | 0/1 [00:00<?, ?it/s]

Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

新聞紙 之 流布 於 寰區 也 香港 則 間 日 呈 奇峙 佳名 於 三秀 滬瀆 則 每 晨 抽 秘鬥彩筆 於 兩 家 或 著錄 應 來復 之 期 教堂 握槧 或 成帙 在 合朔 之 候 京國 傳書 凡 此 紀事 而 纂言 莫不 標新 而 領 異議 分 鄉校 願 考見 夫 濟世 之 經猷 源 討 楹書 願 輯 夫 證 今 之 學問 下 至 方言 里 語 足備 談 諧 雜筆 小詩 亦 供 嗢 噱 奈日力 之 有限 致 篇幅
